# Librerias


In [17]:
import numpy as np
import random
import copy
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances

# Implementacion

In [2]:
class solucion:
  def __init__(self,centros = []):
    self.centros = centros
    self.particion = []
    self.costo = None

In [3]:
def gaussiano(SolAct,delta):
  '''diferencias es un vector con las diferencias
  entre maximo y minimo de cada componente, el cual
  se utiliza para no tener que normalizar los datos '''
  vecina = solucion(copy.deepcopy(SolAct.centros))
  centro = np.random.randint(0,len(SolAct.centros))
  Ga = random.gauss(0,1)
  vecina.centros[centro] += (Ga*delta)
  return(vecina)

In [4]:
def costo(datos,sol):
  '''como usamos SSE debemos calcular la distancia al cuadrado
  entre cada punto y su centroide, y sumar el valor de cada grupo'''
  sse = 0
  for i,centro in enumerate(sol.centros):
    elementos = datos[np.where(sol.particion == i)]
    sse+= np.sum(euclidean_distances(elementos,[centro],squared=True))
  return(sse)

In [5]:
def asignar(datos,centros):
  '''asigna los datos a su centroide mas cercano verificando
  que no haya niguno sin elementos asociados '''
  cambios = True
  while cambios:
    distancias  = euclidean_distances(datos,centros)
    etiquetas = np.argmin(distancias, axis = 1)
    cambios = False
    for i,centro in enumerate(centros):
      if np.argwhere(etiquetas==i).size == 0:
        centros[i] = random.choice(datos)
        cambios=True
  return(etiquetas)

In [6]:
def hipercaja(solucion,datos,utilidades):
  '''esta funcion mueve un centroide de baja utilidad
  a la diagonal de la hipercaja de un grupo con
  alta utilidad'''
  bajaUtilidad = []; AltaUtilidad = [];
  for i,util in enumerate(utilidades):
    if util < 1:
      bajaUtilidad.append(i)
    else:
      AltaUtilidad.append(i)
  cBaja=random.choice(bajaUtilidad) #centride baja utilidad
  cAlta=random.choice(AltaUtilidad) #centroide alta utilidad
  elementosCaja = datos[solucion.particion == cAlta] #elemntosd grupo alta utilidad
  pMax = np.max(elementosCaja,axis =0) #esquina inferior izq hipercaja
  pMin = np.min(elementosCaja,axis =0) #esquina superior derecha hipercaja
  avance = pMax/3 - pMin/3 #vector de tamaño 1/3 de la diagonal
  #movemos los centroides a sus lugares en la diagonal
  solucion.centros[cBaja] = pMin + avance #nueva pos centroide baja U
  solucion.centros[cAlta] = pMax - avance #nueva pos centroide alta U

In [7]:
def calculaDist(solucion,datos):
  distorciones = []
  ng = len(solucion.centros)
  #calculamos la distorcion de todos los grupos
  distancias = euclidean_distances(datos,solucion.centros)
  distorciones = [np.sum(distancias[np.where(solucion.particion == i),i]) for i in range(ng)]
  #calculamos la utilidad
  media = np.mean(distorciones)
  utilidades = [(x/media) for x in distorciones]
  return(utilidades)

In [8]:
def distorcion(sol, datos):
  '''creamos una nueva solucion moviendo un centro de baja
  utilidad cerca de uno de alta utilidad'''
  ng = len(sol.centros)
  vecina = copy.deepcopy(sol)
  #calculamos las utilidades y la media de la distorcion en la sol act
  utilidades = calculaDist(vecina,datos)
  #movemos un centro de baja utilidad cerca de uno de alta utilidad
  hipercaja(vecina,datos,utilidades)
  #reasigamos los datos a su centroide mas cercano
  vecina.particion = asignar(datos,vecina.centros)
  #refinamos los centroides para disminuir la SSE con sus datos
  vecina.centros = np.concatenate([np.mean(
      datos[np.argwhere(vecina.particion==i)],axis=0)for i in range(ng)])
  return(vecina)

In [10]:
def recocido(datos,n_g,T,T_f,T_dist,T_dist_f,alpha,alpha_dist,MaxIter,f_obj):
  '''datos es el conjunto de datos en formato array
  n_g es la cantidad de grupos a formar
  T y T_f son temperatura inicialy final para las distorciones gaussianas
  T_dist y t_dist_f son el maximo y minimo de temperatura para las distorciones de utilidad
  alpha y alpha_dist son los factores de enfriamiento para la temperatura y la temperatura de distorcion
  '''
  indices = np.array(random.sample(range(len(datos)),n_g))
  solActual = solucion(centros=datos[indices])
  solActual.particion = asignar(datos,solActual.centros)
  solActual.costo = costo(datos,solActual)
  best = copy.deepcopy(solActual)
  t_best = T
  while T >= T_f:
    for i in range(MaxIter):
      if solActual.costo < best.costo:
        best = copy.deepcopy(solActual)
        t_best = T
      solVecina = gaussiano(solActual,0.001)
      solVecina.particion = asignar(datos,solVecina.centros)
      solVecina.costo = f_obj(datos,solVecina)
      delta = solVecina.costo-solActual.costo
      if solVecina.costo < solActual.costo or np.e**(-delta/T) > random.random():
        solActual = copy.deepcopy(solVecina)
      if i %20 ==0:
        vecina = distorcion(solActual,datos)
        vecina.particion = asignar(datos,vecina.centros)
        vecina.costo = f_obj(datos,vecina)
        delta2 = vecina.costo-solActual.costo
        if vecina.costo < solActual.costo or np.e**(-delta2/T_dist) > random.random():
          solActual = copy.deepcopy(vecina)
    if T_dist > T_dist_f:
      T_dist *= alpha_dist
    else:
      T_dist = T_dist_f
    T *= alpha
  T = t_best
  solActual = copy.deepcopy(best)
  while T > T_f:
    for i in range(MaxIter):
      solVecina = gaussiano(solActual,0.001)
      solVecina.particion = asignar(datos,solVecina.centros)
      solVecina.costo = f_obj(datos,solVecina)
      delta = solVecina.costo-solActual.costo
      if solVecina.costo < solActual.costo or np.e**(-delta/T) > random.random():
        solActual = copy.deepcopy(solVecina)
    T *= alpha
  return(solActual)